# Running a Molecular Dynamics (MD) simulation of a protein-ligand complex

In this notebook we run an MD simulation of benzene bound to T4-lysozyme L99A.

## On the MD protocol

The plain MD protocol allows the user to run an MD simulation either in solvent or vacuum of e.g. a small molecule, a protein, or a protein-ligand complex. Running an MD simulations can be useful for a variety of things, such as pre-equilibration of a protein-ligand complex prior to running free energy calculations or for getting insights into the general dynamics of the system of interest.

The MD protocol uses different software tools in a series of steps and provides multiple outputs for the user:

| **Step**                                            | **Software used**                  | **Outputs (with default names)**                           |
|:----------------------------------------------------|:-----------------------------------|:-----------------------------------------------------------|
| 1. Input handling using gufe                        | OpenFE, Gufe, RDKit                ||
| 2. Parameterization using OpenMMForceFields & OpenFF| OpenFE - OpenMMForceFields - OpenFF| Forcefield cache (`db.json`) |
| 3. OpenMM object creation                           | OpenFE - OpenMM + OpenMMTools      | Structure of the full system (`system.pdb`) |
| 4. Minimization                                     | OpenFE - OpenMM + OpenMMTools      | Minimized Structure (`minimized.pdb`) |
| 5. NVT equilibration (if not gas phase)             | OpenFE - OpenMM + OpenMMTools      | NVT equilibrated structure (`equil_nvt.pdb`) |
| 6. NPT equilibration (if not gas phase)             | OpenFE - OpenMM + OpenMMTools      | NPT equilibrated structure (`equil_npt.pdb`) |
| 7. NPT production (if not gas phase)                | OpenFE - OpenMM + OpenMMTools      | Simulation trajectory (`simulation.xtc`), Checkpoint file (`checkpoint.chk`), Log output (`simulation.log`) |


## 1. Defining the ChemicalSystem

`ChemicalSystems` are OpenFE containers which define the various components which exist in a system of interest. 
Here, we will be passing the `SmallMoleculeComponent` for benzene, a `ProteinComponent` generated from a PDB file, and a `SolventComponent` which will contain the necessary information for OpenMM’s Modeller class to add water and 0.15 M NaCl around the solute.

In [1]:
import openfe
from openfe import ChemicalSystem, ProteinComponent, SmallMoleculeComponent, SolventComponent
from openff.units import unit

# Define the ligand we are interested in
ligand = SmallMoleculeComponent.from_sdf_file('assets/benzene.sdf')

# Define the solvent environment and protein structure
solvent = SolventComponent(ion_concentration=0.15 * unit.molar)
protein = ProteinComponent.from_pdb_file('assets/t4_lysozyme.pdb', name='t4-lysozyme')

# create the ChemicalSystem
system = ChemicalSystem({'ligand': ligand, 'protein': protein, 'solvent': solvent}, name=f"{ligand.name}_{protein.name}")

## 2. Defining the MD simulation settings

There are various different parameters which can be set to determine how the MD simulation will take place. To allow for maximum user flexibility, these are defined as a series of settings objects which control the following:

| **Setting**                    | **Description**                                                |
|:------------------------------|:-----------------------------------------------------------|
| `simulation_settings`        |  Parameters controlling the simulation plan, including the number of `minimization_steps`, the length of the NVT and NPT equilibration (`equilibration_length_nvt` and `equilibration_length`), and the length of the production MD run (`production_length`). |
| `output_settings`        | Parameters controlling the output from the MD simulations, including file names to save the system after minimization, NVT and NPT equilibration, and production run. Special `output_indices` can be defined to select which portions of the system should be saved (default: `not water`). A `trajectory_write_interval` determines the frequency of writing frames to the output trajectory. |
|  `forcefield_settings` | Settings that define the forcefield for the components, including the general `forcefields`, the `small_molecule_forcefield`, the `nonbonded_method`, and the `nonbonded_cutoff`.  |
| `engine_settings`  | Parameters determining how the OpenMM engine will execute the simulation. This controls the `compute_platform` which will be used to carry out the simulation.  |
| `integrator_settings`  |  Parameters controlling the LangevinSplittingDynamicsMove integrator used for simulation, as well as the `barostat_frequency`. |
| `partial_charge_settings`  | Settings that define which method is used for assigning partial charges.  |
|  `protocol_repeats` |  Defines how often to run the MD protocol. |
| `solvation_settings`  | Parameters to control the `solvent_model` and the `solvent_padding`.  |
|  `thermo_settings` |  Parameters to control e.g. the `temperature` and the `pressure` of the system. |

The easiest way to access and change settings is by first importing the default settings, printing them and then changing the settings according to the user's needs.

In [2]:
from openfe.protocols.openmm_md.plain_md_methods import PlainMDProtocol
from openff.units import unit

settings = PlainMDProtocol.default_settings()
settings.simulation_settings.equilibration_length_nvt = 0.01 * unit.nanosecond # setting the nvt equilibration length to 10 ps
settings.simulation_settings.equilibration_length = 0.01 * unit.nanosecond # setting the npt equilibration length to 10 ps
settings.simulation_settings.production_length = 0.01 * unit.nanosecond # setting the npt production length to 10 ps
settings.engine_settings.compute_platform = 'CPU' # running the simulation on the cpu

In [3]:
settings

{'engine_settings': {'compute_platform': 'CPU'},
 'forcefield_settings': {'constraints': 'hbonds',
                         'forcefields': ['amber/ff14SB.xml',
                                         'amber/tip3p_standard.xml',
                                         'amber/tip3p_HFE_multivalent.xml',
                                         'amber/phosaa10.xml'],
                         'hydrogen_mass': 3.0,
                         'nonbonded_cutoff': <Quantity(1.0, 'nanometer')>,
                         'nonbonded_method': 'PME',
                         'rigid_water': True,
                         'small_molecule_forcefield': 'openff-2.0.0'},
 'integrator_settings': {'barostat_frequency': <Quantity(25, 'timestep')>,
                         'constraint_tolerance': 1e-06,
                         'langevin_collision_rate': <Quantity(1.0, '1 / picosecond')>,
                         'n_restart_attempts': 20,
                         'reassign_velocities': False,
                

## 3.  Creating a `Protocol`

The actual simulation is performed by a [`Protocol`](https://docs.openfree.energy/en/stable/guide/models/execution.html#protocols-and-the-execution-model). 

With the `Settings` inspected and adjusted, we can provide these to the `Protocol`. Here, the OpenMM-based MD Protocol is named `PlainMDProtocol`.

In [4]:
# Creating the Protocol
from openfe.protocols.openmm_md.plain_md_methods import PlainMDProtocol
protocol = PlainMDProtocol(settings=settings)

## 4. Creating the `NonTransformation`
Once we have the `ChemicalSystem`s, and the `Protocol`, we can create the `NonTransformation`. `NonTransformation` here simply means that the system is not "transformed" between two end states as is the case in binding free energy calculations.

In [5]:
nontransformation = openfe.NonTransformation(
    system=system,
    protocol=protocol,  # use protocol created above
    name=f"{system.name}",
)

## 5. Running the MD simulation
There are two ways in which you could execute this MD simulation, either using the OpenFE command-line interface (CLI) or the Python API.

**(a) Using the CLI**

We'll write out the transformation to disk, so that it can be run using the `openfe quickrun` command:

In [7]:
import pathlib
# first we create the directory
md_dir = pathlib.Path("md_input")
md_dir.mkdir(exist_ok=True)

# then we write out the transformation
nontransformation.dump(md_dir / f"{nontransformation.name}.json")

You can run the MD simulation from the CLI by using the `openfe quickrun` command. It
takes a transformation JSON as input, and the flags `-o` to give the final
output JSON file and `-d` for the directory where simulation results should be
stored. For example,

```bash
openfe quickrun path/to/nontransformation.json -o results.json -d working-directory
```

where `path/to/nontransformation.json` is the path to one of the files created above (`md_input/benzene_t4-lysozyme.json`).

**(b) Using the Python API**

Alternatively, the MD simulation can be run by executing the `ProtocolDAG`. The `ProtocolDAG` is created using the `protocol.create()` method and requires as input the `ChemicalSystem`. 

Note: we use the ``shared_basedir`` and ``scratch_basedir`` argument of ``execute_DAG`` in order to set the directory where the simulation files are written to.

In [ ]:
import gufe
import pathlib

# Creating the Protocol
protocol = PlainMDProtocol(settings=settings)
# Creating the Protocol DAG
dag = protocol.create(stateA=system, stateB=system, mapping=None)
workdir = pathlib.Path('./')
# Running the MD simulations
dagres = gufe.protocols.execute_DAG(
    dag,
    shared_basedir=workdir,
    scratch_basedir=workdir,
    keep_shared=True, # set this to True to save the outputs
    n_retries=3
)

Following files were created for the MD run. Note that ff multiple repeats of the MD simulation were run (`protocol_repeats` > 1), you will get N full replicates of these output files with N being the number of repeats.

In [9]:
!ls shared_PlainMDProtocolUnit-256e9093d8f3443b9949b95e7ddbb64e_attempt_0/

checkpoint.chk equil_npt.pdb  minimized.pdb  simulation.xtc
db.json        equil_nvt.pdb  simulation.log system.pdb


### Performance consideration for gas phase MD simulations
For gas phase MD simulations, we suggest setting `OPENMM_CPU_THREADS` to `1` to obtain good performance.